In [143]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [144]:
wiki_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [145]:
wiki.name

'wikipedia'

In [146]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018239560370>, search_kwargs={})

In [147]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool.")

In [148]:
retriever_tool.name

'langsmith_search'

In [149]:
## Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(arxiv_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [150]:
tools=[wiki,arxiv,retriever_tool]

In [151]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\User\\OneDrive\\Desktop\\Krish Naik\\LangChain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001826745EE60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], ve

In [152]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral", temperature=0)

In [153]:
from langchain import hub
# Get the prompt to use. You can modify this!
prompt = hub.pull("hwchase17/react")

In [154]:
# Agents
from langchain.agents import create_react_agent
agent=create_react_agent(llm,tools,prompt)

In [155]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018239560370>, searc

In [156]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
 The question is asking for information about LangSmith. To get this information, I will use the `langsmith_search` tool.

Action: langsmith_search
Action Input: 'LangSmith'Get started by creating your first prompt.
Iterate on models and prompts using the Playground.
Manage prompts programmatically in your application.
Was this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.

Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluatio

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform for building production-grade Language Model (LLM) applications. It allows users to closely monitor and evaluate their application, providing observability features such as tracing, metrics, dashboards, and alerts. LangSmith can be used with or without LangChain's open source frameworks langchain and langgraph. It is especially useful for LLMs due to their non-deterministic nature, making them trickier to debug. LangSmith has LLM-native observability features that cover all stages of application development, from prototyping to production."}

In [157]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...


 I need to find information about Machine Learning. The best tool for this is Wikipedia.

Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen I now know the final answer.
Final Answer: Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize it to make predictions or decisions about new data.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize it to make predictions or decisions about new data.'}

In [158]:
agent_executor.invoke({"input":"What is paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
 This question seems to be asking for information about a specific scientific paper on Arxiv.org. I should use the 'arxiv' tool to find this information.

Action: arxiv
Action Input: "1605.08386"Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension. Based on the observation from the 'arxiv' tool, it appears that paper 1605.08386 titled "Heat-bath random walks with Markov bases" discusses the study of graphs on lattice

{'input': 'What is paper 1605.08386 about?',
 'output': 'Paper 1605.08386 is about studying the properties of certain graphs and their heat-bath random walks, focusing on their diameter, mixing behavior, and expansion properties under specific conditions.'}